# Importação

In [1]:
import pandas as pd
import pyomo.environ as pyo 
from pyomo.environ import SolverFactory
from pyomo.environ import Var, Integers, Binary, Constraint,TransformationFactory,RangeSet
from datetime import time, datetime
#import highspy
#import gurobipy

# Timestamp da rodada
P_HoraDaRodada = datetime.now()
P_HoraDaRodadaTimeStamp = int(P_HoraDaRodada.strftime("%Y%m%d%H%M%S"))

# Inicialização do modelo
model = pyo.ConcreteModel()
solver_selecionado = 'appsi_highs'
relaxacao_linear = 0

# importação de libs específicas de acordo com o solver
if solver_selecionado == 'highs':
    import highspy
if solver_selecionado == 'gurobi':
    import gurobipy
if solver_selecionado == 'mosek':
    import mosek

# Inicialização de dicionários
Ps_NomeDaRodada = {}
P_SOL_incumbent_solution = {}
P_SOL_solver_status = {}
P_SOL_solver_time = {}
P_SOL_best_bound = {}
P_SOL_optimality_gap ={}
start_time = {}
end_time = {}
elapsed_time = {}

# Funções de apoio
def Func_ConversaoDatetimeParaMinutosEscalares(P_ARG_Datetime):
    P_Minutos = P_ARG_Datetime.hour * 60 + P_ARG_Datetime.minute
    return P_Minutos

def Func_ConversaoDatetimeParaMinutosParametros(data_dict):
    return {
        key: (val.hour * 60 + val.minute if isinstance(val, time) else val)
        for key, val in data_dict.items()
    }

# Recursão

In [2]:
S_Instancias = ["i09","i10","i10C","i11","i12","i13","i14C","i15","i25","i30","i37","i38","i45","i46C","i50"]
S_InstanciasMuitoPequenas = ["i09","i10","i10C","i11"]
S_InstanciasPequenas = ["i09","i10","i10C","i11","i12","i13"]
S_InstanciasGrandes = ["i25","i30","i37","i38","i45","i46C","i50"]
S_InstanciasRodada = ["i25","i30","i37","i38"]
S_InstanciasTeste = ["i50"]

## Início da recursão #####################################################################################################################
for instancia in S_InstanciasTeste:
    
    # IMPORTAÇÃO ###############################################################################################################################3
    
    #1 Leitura do arquivo .xslx para conjuntos e parâmetros
    Ps_CaminhoDaPlanilha = f"G:\Meu Drive\Planilhas para Estudos\{instancia}.xlsx"
    #1.1 Escalares
    df_escalares = pd.read_excel(Ps_CaminhoDaPlanilha, sheet_name = "Escalares")
    #print(df_escalares)
    #print(df_escalares.dtypes)

    #1.2 Programacao Inicial
    df_proginicial = pd.read_excel(Ps_CaminhoDaPlanilha, sheet_name = "ProgramacaoInicial")
    #print(df_proginicial)
    #print(df_proginicial.dtypes)

    #1.3 Aeronaves
    df_aeronaves = pd.read_excel(Ps_CaminhoDaPlanilha, sheet_name = "Aeronaves")
    #print(df_aeronaves)
    #print(df_aeronaves.dtypes)

    #1.1 Compatibilidade
    df_compatibilidade = pd.read_excel(Ps_CaminhoDaPlanilha, sheet_name = "Compatibilidade")
    #print(df_compatibilidade)
    #print(df_compatibilidade.dtypes)

    ########################################################## Conjuntos
    # 2 Criação de conjuntos e parametros
    # 2.1 Conjuntos

    #Voos
    # Todos
    model.S_VoosReais = pyo.Set(initialize=df_proginicial.Voo)
    S_VoosReais = model.S_VoosReais
    #print(list(S_VoosReais))
    # Comitiva
    model.S_VoosComitiva = pyo.Set(
        within=model.S_VoosReais, 
        initialize=df_proginicial[df_proginicial['TipoDoVoo'] == "Tab.Comitivas"]['Voo'].tolist()
    )
    S_VoosComitiva = model.S_VoosComitiva
    # Transferência 2
    model.S_VoosT2 = pyo.Set(
        within=model.S_VoosReais, 
        initialize=df_proginicial[df_proginicial['TipoDoVoo'] == "Transf.2"]['Voo'].tolist()
    )
    S_VoosT2 = model.S_VoosT2
    # Transferência 1
    model.S_VoosT1 = pyo.Set(
        within=model.S_VoosReais, 
        initialize=df_proginicial[df_proginicial['TipoDoVoo'] == "Transf.1"]['Voo'].tolist()
    )
    S_VoosT1 = model.S_VoosT1
    # Transferência todos
    S_VoosTransferencia = S_VoosT1.union(S_VoosT2)
    # Tabela
    model.S_VoosTabela = pyo.Set(
        within=model.S_VoosReais, 
        initialize=df_proginicial[df_proginicial['TipoDoVoo'] == "Tab."]['Voo'].tolist()
    )
    S_VoosTabela = model.S_VoosTabela
    # Tabela + Comitiva
    S_VoosTabelaComitivaLista = S_VoosTabela.union(S_VoosComitiva)
    model.S_VoosTabelaComitiva = pyo.Set(
        initialize=S_VoosTabelaComitivaLista
    )
    S_VoosTabelaComitiva = model.S_VoosTabelaComitiva
    print(list(S_VoosTabelaComitiva))

    # Fictícios
    # Como esse modelo usa uma abordagem de routing, é necessário criar o ponto de partida e de fim do modelo;
    # nesse caso, são o voo 0 e o voo card(S_Voos)+1
    S_ListaVoosFicticios = [0, len(df_proginicial.Voo)+1]
    model.S_VoosFicticios = pyo.Set(initialize=S_ListaVoosFicticios)
    S_VoosFicticios = model.S_VoosFicticios
    #print(list(S_VoosFicticios))

    # Unindo conjuntos de voos reais e fictícios em um superconjunto
    S_VoosTodosLista = S_VoosReais.union(S_VoosFicticios)
    model.S_VoosTodos = pyo.Set( 
        initialize=S_VoosTodosLista
    )
    S_VoosTodos =model.S_VoosTodos 
    #print("S_VoosTodos", list(S_VoosTodos))

    # Outros subconjuntos de voos usados no modelo
    # Reais + voo 0
    # a função sort se faz necessária para que o conjunto venha ordenado e o código tenha um comportamento determinístico
    S_ListaVoosReaisEZero = sorted({S_VoosFicticios.at(1)}.union(S_VoosReais))
    model.S_VoosReaisEZero = pyo.Set(initialize=S_ListaVoosReaisEZero)
    S_VoosReaisEZero = model.S_VoosReaisEZero
    # Reais + voo final
    S_ListaVoosReaisEFinal= (S_VoosReais.union(sorted({S_VoosFicticios.at(len(S_VoosFicticios))})))
    model.S_VoosReaisEFinal = pyo.Set(initialize=S_ListaVoosReaisEFinal)
    S_VoosReaisEFinal = model.S_VoosReaisEFinal
    #print("S_VoosReaisEFinal=", list(S_VoosReaisEFinal))
    # Reais exceto voos comitiva
    S_VoosReaisSemComitiva= S_VoosReais - S_VoosComitiva
    #print("S_VoosReaisSemComitiva=", list(S_VoosReaisSemComitiva))

    #Unidades Marítimas
    model.S_UM = pyo.Set(initialize=df_proginicial.UMDestino)
    S_UM = model.S_UM

    #Aeronaves
    #Geral
    model.S_Aeronaves = pyo.Set(initialize=df_aeronaves.Aeronave)
    S_Aeronaves = model.S_Aeronaves 
    # Spot
    model.S_AeronavesSpot= pyo.Set(
        within=model.S_Aeronaves, 
        initialize=df_aeronaves[df_aeronaves['TipoDaAeronave'] == 3]['Aeronave'].tolist()
    )
    S_AeronavesSpot = model.S_AeronavesSpot
    # Pool
    model.S_AeronavesPool= pyo.Set(
        within=model.S_Aeronaves, 
        initialize=df_aeronaves[df_aeronaves['TipoDaAeronave'] == 2]['Aeronave'].tolist()
    )
    S_AeronavesPool = model.S_AeronavesPool
    # Comum
    model.S_AeronavesComum= pyo.Set(
        within=model.S_Aeronaves, 
        initialize=df_aeronaves[df_aeronaves['TipoDaAeronave'] == 1]['Aeronave'].tolist()
    )
    S_AeronavesComum = model.S_AeronavesComum

    ##### Parâmetros

    #2.2.1 Escalares usando param
    Ps_NomeDaInstancia = df_escalares.NomeDaInstancia[0]  
    print("Ps_NomeDaInstancia = ",Ps_NomeDaInstancia)

    P_NumeroDeVoos= df_escalares.NumeroDeVoos[0]
    #print("P_NumeroDeVoos = ",P_NumeroDeVoos)

    P_TempoDeSeguranca = df_escalares.TempoDeSeguranca[0]
    #print("P_TempoDeSeguranca = ",P_TempoDeSeguranca)

    P_NumeroMaxDeVoosPorAeronave = df_escalares.NumeroMaxDeVoosPorAeronave[0]
    #print("P_NumeroMaxDeVoosPorAeronave = ",P_NumeroMaxDeVoosPorAeronave)

    P_TurnaroundTime = df_escalares.TurnaroundTime[0]
    #print("P_TurnaroundTime = ",P_TurnaroundTime)

    P_AtrasoLimite1 = df_escalares.AtrasoLimite1[0]  
    #print("P_AtrasoLimite1 = ",P_AtrasoLimite1)

    P_AtrasoLimite2 = df_escalares.AtrasoLimite2[0]  
    #print("P_AtrasoLimite2 = ",P_AtrasoLimite2)

    P_InicioDoHorizonteDatetime = df_escalares.InicioDoHorizonte[0]
    #print("P_InicioDoHorizonteDatetime = ",P_InicioDoHorizonteDatetime)
    P_InicioDoHorizonte = Func_ConversaoDatetimeParaMinutosEscalares(P_InicioDoHorizonteDatetime)
    #print("P_InicioDoHorizonte = ",P_InicioDoHorizonte)

    P_FimDoHorizonteDatetime = df_escalares.FimDoHorizonte[0]
    #print("P_FimDoHorizonteDatetime = ",P_FimDoHorizonteDatetime)
    P_FimDoHorizonte = Func_ConversaoDatetimeParaMinutosEscalares(P_FimDoHorizonteDatetime)
    #print("P_FimDoHorizonte = ",P_FimDoHorizonte)

    P_PenCancelamentoComitiva = df_escalares.PenCancelamentoComitiva[0] 
    #print("P_PenCancelamentoComitiva = ",P_PenCancelamentoComitiva.value)

    P_PenCancelamento2Dias = df_escalares.PenCancelamento2Dias[0]
    #print("P_PenCancelamento2Dias = ",P_PenCancelamento2Dias.value)

    P_PenCancelamento1Dia = df_escalares.PenCancelamento1Dia[0]
    #print("P_PenCancelamento1Dia = ",P_PenCancelamento1Dia.value)

    P_PenCancelamentoTabela = df_escalares.PenCancelamentoTabela[0] 
    #print("P_PenCancelamentoTabela = ",P_PenCancelamentoTabela.value)

    P_PenUsoAeronaveSpot = df_escalares.PenUsoAeronaveSpot[0]
    #print("P_PenUsoAeronaveSpot = ",P_PenUsoAeronaveSpot.value)

    P_PenUsoAeronavePool = df_escalares.PenUsoAeronavePool[0]
    #print("P_PenUsoAeronavePool = ",P_PenUsoAeronavePool.value)

    P_PenUsoAeronaveComum = df_escalares.PenUsoAeronaveComum[0] 
    #print("P_PenUsoAeronaveComum = ",P_PenUsoAeronaveComum.value)

    P_PenTrocaAeronave = df_escalares.PenTrocaAeronave[0] 
    #print("P_PenTrocaAeronave = ",P_PenTrocaAeronave.value)

    P_PenAtrasoTipo2 = df_escalares.PenAtrasoTipo2[0]
    #print("P_PenAtrasoTipo2 = ",P_PenAtrasoTipo2.value)

    P_PenAtrasoTipo1 = df_escalares.PenAtrasoTipo1[0]
    #print("P_PenAtrasoTipo1 = ",P_PenAtrasoTipo1.value)

    P_PenAtrasoMinuto = df_escalares.PenAtrasoMinuto[0]
    #print("P_PenAtrasoMinuto = ",P_PenAtrasoMinuto.value)

    #2.2.2 Indexados
    #Convertendo para um dicionário, mas o índice deve ser igual ao nome da coluna
    #Horário Inicial
    #   Conversao de datetime para minutos
    P_HorarioInicialDatetime_dict = df_proginicial.set_index('Voo')['HorarioInicial'].to_dict()
    P_HorarioInicial_dict = Func_ConversaoDatetimeParaMinutosParametros(P_HorarioInicialDatetime_dict)
    #   Alteração do horário inicial para ser sempre 07:00
    for voo in S_VoosReais:
        if P_HorarioInicial_dict[voo] < 420:
            P_HorarioInicial_dict[voo] = 420

    #   Transformação de lista em parâmetro para modelo
    model.P_HorarioInicial = pyo.Param(S_VoosReais, initialize=P_HorarioInicial_dict)
    P_HorarioInicial = model.P_HorarioInicial
    #print("P_HorarioInicial = ",P_HorarioInicial.display())

    #UM Destino
    P_UMDestino_dict = df_proginicial.set_index('Voo')['UMDestino'].to_dict()
    model.P_UMDestino = pyo.Param(S_VoosReais, initialize=P_UMDestino_dict)
    P_UMDestino = model.P_UMDestino  
    #print("P_UMDestino = ",P_UMDestino.display())

    #Duração do Voo
    #   Conversao de datetime para minutos
    P_DuracaoDoVooDatetime_dict = df_proginicial.set_index('Voo')['DuracaoDoVoo'].to_dict()
    P_DuracaoDoVoo_dict = Func_ConversaoDatetimeParaMinutosParametros(P_DuracaoDoVooDatetime_dict)
    #   Transformação de lista em parâmetro para modelo
    model.P_DuracaoDoVoo = pyo.Param(S_VoosReais, initialize=P_DuracaoDoVoo_dict)
    P_DuracaoDoVoo = model.P_DuracaoDoVoo  
    print("P_DuracaoDoVoo= ",P_DuracaoDoVoo.display())

    # Aeronave inicial
    P_AeronaveInicialString_dict = df_proginicial.set_index('Voo')['AeronaveInicial'].to_dict()
    # Convertendo para um parâmetro de 2 índices binário, em formato de dicionário
    P_AeronaveInicial = {}
    for i in S_VoosReais:
        for h in S_Aeronaves:
            if P_AeronaveInicialString_dict[i] == h:
                P_AeronaveInicial[i,h] = 1
            else:
                P_AeronaveInicial[i,h] = 0

    # Declarando o dicionário como parâmetro         
    model.P_AeronaveInicial = pyo.Param(S_VoosReais, S_Aeronaves, initialize=P_AeronaveInicial)
    P_AeronaveInicial = model.P_AeronaveInicial
    #print("P_AeronaveInicial = ",P_AeronaveInicial.display())

    # Tempo de Permanência
    #   Conversao de datetime para minutos
    P_TempoDePermanenciaDatetime_dict = df_proginicial.set_index('Voo')['TempoDePermanencia'].to_dict()
    P_TempoDePermanencia_dict = Func_ConversaoDatetimeParaMinutosParametros(P_TempoDePermanenciaDatetime_dict)
    #   Transformação de lista em parâmetro para modelo
    model.P_TempoDePermanencia = pyo.Param(S_VoosReais, initialize=P_TempoDePermanencia_dict)
    P_TempoDePermanencia = model.P_TempoDePermanencia 
    #print("P_TempoDePermanencia = ",P_TempoDePermanencia.display())

    # Tipo de Aeronave
    P_TipoDeAeronave_dict = df_aeronaves.set_index('Aeronave')['TipoDaAeronave'].to_dict()
    model.P_TipoDeAeronave = pyo.Param(S_Aeronaves, initialize=P_TipoDeAeronave_dict)
    P_TipoDeAeronave = model.P_TipoDeAeronave 
    #print("P_TipoDeAeronave = ",P_TipoDeAeronave.display())

    # Compatibilidade
    # Como esse parâmetro possui mais de um índice, é necessária uma manipulação prévia
    # Além disso, o dataframe com os dados deste parâmetro está num formato em que um dos indices (S_Voos) está na coluna e o outro (S_Aeronaves) está como cabeçalho
    # Para isso, os passos abaixo são seguidos:
    # Passo 1) usar a função .melt para alterar o formato do dataframe, transferindo o cabeçalho para uma nova coluna:
    df_compatibilidade_mod= df_compatibilidade.melt(id_vars=['Voo'], var_name='Aeronaves', value_name='Compatibilidade')
    # Passo 2) Create the dictionary in the (S_VoosReais, S_UM): Value format, que é o formato que a função pyo.Param aceita os dados de um parâmetro multiíndice
    P_Compatibilidade_dict = {(row['Voo'], row['Aeronaves']): row['Compatibilidade'] for _, row in df_compatibilidade_mod.iterrows()}
    # Passo 3) Por fim, realizar a atribuição ao parâmetro normalmente
    model.P_Compatibilidade = pyo.Param(S_VoosReais, S_Aeronaves, initialize=P_Compatibilidade_dict)
    P_Compatibilidade = model.P_Compatibilidade 
    # print("P_Compatibilidade = ",P_Compatibilidade.display())

    #2.2.1 Escalares usando param
    Ps_NomeDaInstancia = df_escalares.NomeDaInstancia[0]  
    print("Ps_NomeDaInstancia = ",Ps_NomeDaInstancia)

    P_NumeroDeVoos= df_escalares.NumeroDeVoos[0]
    #print("P_NumeroDeVoos = ",P_NumeroDeVoos)

    P_TempoDeSeguranca = df_escalares.TempoDeSeguranca[0]
    #print("P_TempoDeSeguranca = ",P_TempoDeSeguranca)

    P_NumeroMaxDeVoosPorAeronave = df_escalares.NumeroMaxDeVoosPorAeronave[0]
    #print("P_NumeroMaxDeVoosPorAeronave = ",P_NumeroMaxDeVoosPorAeronave)

    P_TurnaroundTime = df_escalares.TurnaroundTime[0]
    #print("P_TurnaroundTime = ",P_TurnaroundTime)

    P_AtrasoLimite1 = df_escalares.AtrasoLimite1[0]  
    #print("P_AtrasoLimite1 = ",P_AtrasoLimite1)

    P_AtrasoLimite2 = df_escalares.AtrasoLimite2[0]  
    #print("P_AtrasoLimite2 = ",P_AtrasoLimite2)

    P_InicioDoHorizonteDatetime = df_escalares.InicioDoHorizonte[0]
    #print("P_InicioDoHorizonteDatetime = ",P_InicioDoHorizonteDatetime)
    P_InicioDoHorizonte = Func_ConversaoDatetimeParaMinutosEscalares(P_InicioDoHorizonteDatetime)
    #print("P_InicioDoHorizonte = ",P_InicioDoHorizonte)

    P_FimDoHorizonteDatetime = df_escalares.FimDoHorizonte[0]
    #print("P_FimDoHorizonteDatetime = ",P_FimDoHorizonteDatetime)
    P_FimDoHorizonte = Func_ConversaoDatetimeParaMinutosEscalares(P_FimDoHorizonteDatetime)
    #print("P_FimDoHorizonte = ",P_FimDoHorizonte)

    P_PenCancelamentoComitiva = df_escalares.PenCancelamentoComitiva[0] 
    #print("P_PenCancelamentoComitiva = ",P_PenCancelamentoComitiva.value)

    P_PenCancelamento2Dias = df_escalares.PenCancelamento2Dias[0]
    #print("P_PenCancelamento2Dias = ",P_PenCancelamento2Dias.value)

    P_PenCancelamento1Dia = df_escalares.PenCancelamento1Dia[0]
    #print("P_PenCancelamento1Dia = ",P_PenCancelamento1Dia.value)

    P_PenCancelamentoTabela = df_escalares.PenCancelamentoTabela[0] 
    #print("P_PenCancelamentoTabela = ",P_PenCancelamentoTabela.value)

    P_PenUsoAeronaveSpot = df_escalares.PenUsoAeronaveSpot[0]
    #print("P_PenUsoAeronaveSpot = ",P_PenUsoAeronaveSpot.value)

    P_PenUsoAeronavePool = df_escalares.PenUsoAeronavePool[0]
    #print("P_PenUsoAeronavePool = ",P_PenUsoAeronavePool.value)

    P_PenUsoAeronaveComum = df_escalares.PenUsoAeronaveComum[0] 
    #print("P_PenUsoAeronaveComum = ",P_PenUsoAeronaveComum.value)

    P_PenTrocaAeronave = df_escalares.PenTrocaAeronave[0] 
    #print("P_PenTrocaAeronave = ",P_PenTrocaAeronave.value)

    P_PenAtrasoTipo2 = df_escalares.PenAtrasoTipo2[0]
    #print("P_PenAtrasoTipo2 = ",P_PenAtrasoTipo2.value)

    P_PenAtrasoTipo1 = df_escalares.PenAtrasoTipo1[0]
    #print("P_PenAtrasoTipo1 = ",P_PenAtrasoTipo1.value)

    P_PenAtrasoMinuto = df_escalares.PenAtrasoMinuto[0]
    #print("P_PenAtrasoMinuto = ",P_PenAtrasoMinuto.value)

    #2.2.2 Indexados
    #Convertendo para um dicionário, mas o índice deve ser igual ao nome da coluna
    #Horário Inicial
    #   Conversao de datetime para minutos
    P_HorarioInicialDatetime_dict = df_proginicial.set_index('Voo')['HorarioInicial'].to_dict()
    P_HorarioInicial_dict = Func_ConversaoDatetimeParaMinutosParametros(P_HorarioInicialDatetime_dict)
    #   Alteração do horário inicial para ser sempre 07:00
    for voo in S_VoosReais:
        if P_HorarioInicial_dict[voo] < 420:
            P_HorarioInicial_dict[voo] = 420

    #   Transformação de lista em parâmetro para modelo
    model.P_HorarioInicial = pyo.Param(S_VoosReais, initialize=P_HorarioInicial_dict)
    P_HorarioInicial = model.P_HorarioInicial
    #print("P_HorarioInicial = ",P_HorarioInicial.display())

    #UM Destino
    P_UMDestino_dict = df_proginicial.set_index('Voo')['UMDestino'].to_dict()
    model.P_UMDestino = pyo.Param(S_VoosReais, initialize=P_UMDestino_dict)
    P_UMDestino = model.P_UMDestino  
    #print("P_UMDestino = ",P_UMDestino.display())

    #Duração do Voo
    #   Conversao de datetime para minutos
    P_DuracaoDoVooDatetime_dict = df_proginicial.set_index('Voo')['DuracaoDoVoo'].to_dict()
    P_DuracaoDoVoo_dict = Func_ConversaoDatetimeParaMinutosParametros(P_DuracaoDoVooDatetime_dict)
    #   Transformação de lista em parâmetro para modelo
    model.P_DuracaoDoVoo = pyo.Param(S_VoosReais, initialize=P_DuracaoDoVoo_dict)
    P_DuracaoDoVoo = model.P_DuracaoDoVoo  
    #print("P_DuracaoDoVoo= ",P_DuracaoDoVoo.display())

    # Aeronave inicial
    P_AeronaveInicialString_dict = df_proginicial.set_index('Voo')['AeronaveInicial'].to_dict()
    # Convertendo para um parâmetro de 2 índices binário, em formato de dicionário
    P_AeronaveInicial = {}
    for i in S_VoosReais:
        for h in S_Aeronaves:
            if P_AeronaveInicialString_dict[i] == h:
                P_AeronaveInicial[i,h] = 1
            else:
                P_AeronaveInicial[i,h] = 0

    # Declarando o dicionário como parâmetro         
    model.P_AeronaveInicial = pyo.Param(S_VoosReais, S_Aeronaves, initialize=P_AeronaveInicial)
    P_AeronaveInicial = model.P_AeronaveInicial
    #print("P_AeronaveInicial = ",P_AeronaveInicial.display())

    # Tempo de Permanência
    #   Conversao de datetime para minutos
    P_TempoDePermanenciaDatetime_dict = df_proginicial.set_index('Voo')['TempoDePermanencia'].to_dict()
    P_TempoDePermanencia_dict = Func_ConversaoDatetimeParaMinutosParametros(P_TempoDePermanenciaDatetime_dict)
    #   Transformação de lista em parâmetro para modelo
    model.P_TempoDePermanencia = pyo.Param(S_VoosReais, initialize=P_TempoDePermanencia_dict)
    P_TempoDePermanencia = model.P_TempoDePermanencia 
    #print("P_TempoDePermanencia = ",P_TempoDePermanencia.display())

    # Tipo de Aeronave
    P_TipoDeAeronave_dict = df_aeronaves.set_index('Aeronave')['TipoDaAeronave'].to_dict()
    model.P_TipoDeAeronave = pyo.Param(S_Aeronaves, initialize=P_TipoDeAeronave_dict)
    P_TipoDeAeronave = model.P_TipoDeAeronave 
    #print("P_TipoDeAeronave = ",P_TipoDeAeronave.display())

    # Compatibilidade
    # Como esse parâmetro possui mais de um índice, é necessária uma manipulação prévia
    # Além disso, o dataframe com os dados deste parâmetro está num formato em que um dos indices (S_Voos) está na coluna e o outro (S_Aeronaves) está como cabeçalho
    # Para isso, os passos abaixo são seguidos:
    # Passo 1) usar a função .melt para alterar o formato do dataframe, transferindo o cabeçalho para uma nova coluna:
    df_compatibilidade_mod= df_compatibilidade.melt(id_vars=['Voo'], var_name='Aeronaves', value_name='Compatibilidade')
    # Passo 2) Create the dictionary in the (S_VoosReais, S_UM): Value format, que é o formato que a função pyo.Param aceita os dados de um parâmetro multiíndice
    P_Compatibilidade_dict = {(row['Voo'], row['Aeronaves']): row['Compatibilidade'] for _, row in df_compatibilidade_mod.iterrows()}
    # Passo 3) Por fim, realizar a atribuição ao parâmetro normalmente
    model.P_Compatibilidade = pyo.Param(S_VoosReais, S_Aeronaves, initialize=P_Compatibilidade_dict)
    P_Compatibilidade = model.P_Compatibilidade 
    # print("P_Compatibilidade = ",P_Compatibilidade.display())

    ##################################################################################################### Variáveis e FO
    # Variáveis do modelo
    # X(i,j,h)
    model.V_X = pyo.Var(S_VoosTodos, S_VoosTodos, S_Aeronaves, domain = pyo.Binary, bounds = [0,1])
    V_X = model.V_X 
    #Y(i,h)
    model.V_Y = pyo.Var(S_VoosReais, S_Aeronaves, domain = pyo.Binary, bounds = [0,1])
    V_Y = model.V_Y 
    #Z(i,j)
    model.V_Z = pyo.Var(S_VoosReais, S_VoosReais, domain = pyo.Binary, bounds = [0,1])
    V_Z = model.V_Z
    #V(h)
    model.V_V = pyo.Var(S_Aeronaves, domain = pyo.Binary, bounds = [0,1])
    V_V= model.V_V
    #BI(i)
    model.V_BI = pyo.Var(S_VoosReais, domain = pyo.Binary, bounds = [0,1])
    V_BI = model.V_BI
    #BII(i)
    model.V_BII = pyo.Var(S_VoosReais, domain = pyo.Binary, bounds = [0,1])
    V_BII = model.V_BII
    #DT(i)
    model.V_DT = pyo.Var(S_VoosTodos, domain = pyo.NonNegativeReals)
    V_DT = model.V_DT
    #AT(i)
    model.V_AT = pyo.Var(S_VoosTodos, domain = pyo.NonNegativeReals)
    V_AT = model.V_AT
    #D(i)
    model.V_D = pyo.Var(S_VoosTodos, domain = pyo.NonNegativeReals)
    V_D = model.V_D

    #Funcao Objetivo
    V_FO = (
            P_PenCancelamentoComitiva * (len(S_VoosComitiva) - sum(V_Y[vooC,h] for vooC in S_VoosComitiva for h in S_Aeronaves))
        +P_PenCancelamento2Dias * (len(S_VoosT2) - sum(V_Y[vooT2,h] for vooT2 in S_VoosT2 for h in S_Aeronaves))
        +P_PenCancelamento1Dia * (len(S_VoosT1) - sum(V_Y[vooT1,h] for vooT1 in S_VoosT1 for h in S_Aeronaves))
        +P_PenCancelamentoTabela * (len(S_VoosTabela) - sum(V_Y[vooTabela,h] for vooTabela in S_VoosTabela for h in S_Aeronaves))
        +P_PenUsoAeronaveSpot * sum(V_V[hSpot] for hSpot in S_AeronavesSpot)
        +P_PenUsoAeronavePool * sum(V_V[hPool] for hPool in S_AeronavesPool)
        +P_PenUsoAeronaveComum * sum(V_V[hComum] for hComum in S_AeronavesComum)
        +P_PenAtrasoTipo2 * sum(V_BII[vooTabelaComitiva] for vooTabelaComitiva in S_VoosTabelaComitiva)
        +P_PenAtrasoTipo1 * sum(V_BI[vooTabelaComitiva] for vooTabelaComitiva in S_VoosTabelaComitiva)
        +P_PenAtrasoMinuto * sum(V_D[vooTodos] for vooTodos in S_VoosTodos)
        +P_PenTrocaAeronave * sum(V_Y[vooTabelaComitiva,h] * (1 - P_AeronaveInicial[vooTabelaComitiva,h]) for vooTabelaComitiva in S_VoosTabelaComitiva for h in S_Aeronaves)
    )

    model.obj = pyo.Objective(expr = V_FO, sense = pyo.minimize)

    # Restrições ################################################################################################
    # 4.2 C_ConservacaoDoFluxo - ok - fc2
    model.C_ConservacaoDoFluxo = pyo.Constraint(S_VoosTodos, S_Aeronaves,
                                                rule = lambda model, i,h: 
                                                    sum(V_X[j,i,h] for j in S_VoosReaisEZero if i != j)
                                                    ==
                                                    sum(V_X[i,j,h] for j in S_VoosReaisEFinal if i != j)) 

    # 4.3 C_ArcosEntradaESaida - ok -fc3
    model.C_ArcosEntradaESaida = pyo.Constraint(rule = lambda model: 
                                                sum(V_X[S_VoosFicticios.at(1),j,h] for j in S_VoosReais for h in S_Aeronaves)
                                                    ==
                                                sum(V_X[i,S_VoosFicticios.at(len(S_VoosFicticios)),h] for i in S_VoosReais for h in S_Aeronaves))

    # 4.4 C_NumeroDeAeronavesUtilizadas - ok - fc4
    model.C_NumeroDeAeronavesUtilizadas = pyo.Constraint(S_Aeronaves,
                                                        rule = lambda model, h: 
                                                        sum(V_X[S_VoosFicticios.at(1),j,h] for j in S_VoosReais)
                                                            ==
                                                        V_V[h])             

    # 4.5 C_RelacaoDesignacaoEPrecedencia - ok - fc1
    model.C_RelacaoDesignacaoEPrecedencia = pyo.Constraint(S_VoosReais, S_Aeronaves,
                                                        rule = lambda model, i,h: 
                                                        sum(V_X[i,j,h] for j in S_VoosReaisEFinal if i != j)
                                                            ==
                                                        V_Y[i,h])
    # 4.6 C_MaximoUmVooPorAeronave - ok - ac1
    model.C_MaximoUmVooPorAeronave = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        sum(V_Y[i,h] for h in S_Aeronaves)
                                                            <=
                                                        1)
    # 4.7 C_RelacaoDesignacaoEUsoSuperior - ok - ac2
    model.C_RelacaoDesignacaoEUsoSuperior = pyo.Constraint(S_VoosReais,S_Aeronaves,
                                                        rule = lambda model, i,h: 
                                                        V_Y[i,h]
                                                            <=
                                                        V_V[h])
    # 4.8 C_MaximoDeVoos - ok - ac3
    model.C_MaximoDeVoos = pyo.Constraint(S_Aeronaves,
                                                        rule = lambda model, h: 
                                                        sum(V_Y[i,h] for i in S_VoosReais)
                                                        <=
                                                        P_NumeroMaxDeVoosPorAeronave * V_V[h])
    # 4.9 C_RelacaoDesignacaoEUsoInferior - ok -ac4
    model.C_RelacaoDesignacaoEUsoInferior = pyo.Constraint(S_Aeronaves,
                                                        rule = lambda model, h: 
                                                        sum(V_Y[i,h] for i in S_VoosReais)
                                                            >=
                                                            V_V[h])
    # 4.10 C_BloqueioUMPorVooComitiva
    model.C_BloqueioUMPorVooComitiva = pyo.Constraint(S_VoosComitiva,S_VoosReaisSemComitiva,S_Aeronaves,
                                                        rule = lambda model, i,j,h: 
                                                        V_Y[i,h] + V_Y[j,h] - P_FimDoHorizonte*(1 - V_Z[i,j])
                                                        <=
                                                        1)
    # 4.11 C_SequenciaCronologica - ok - tc1
    model.C_SequenciaCronologica = pyo.Constraint(S_VoosReaisEZero, S_VoosReaisEFinal,
                                                        rule = lambda model, i,j: 
                                                        V_DT[j]
                                                            >=
                                                        V_AT[i] 
                                                        + P_TurnaroundTime*sum(V_X[i,j,h] for h in S_Aeronaves) 
                                                        - P_FimDoHorizonte * (1 - sum(V_X[i,j,h] for h in S_Aeronaves))
                                                        if i != j else pyo.Constraint.Skip)
    # 4.12 C_HorarioDeChegada - ok - tc2
    model.C_HorarioDeChegada = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        V_AT[i]
                                                            ==
                                                        V_DT[i] + P_DuracaoDoVoo[i]*sum(V_Y[i,h] for h in S_Aeronaves))
    # 4.13 C_PartidaAposOHorarioPrevisto - ok - tc3
    model.C_PartidaAposOHorarioPrevisto = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        P_HorarioInicial[i] * sum(V_Y[i,h] for h in S_Aeronaves)
                                                            <=
                                                        V_DT[i])
    # 4.14 C_DeterminaValorDePartida - ok - tc4
    model.C_DeterminaValorDePartida = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        V_DT[i]
                                                            <=
                                                        P_HorarioInicial[i] * sum(V_Y[i,h] for h in S_Aeronaves) + V_D[i])
    # 4.15 C_ChegadaDentroDaJanelaDeTempo - ok -tc5
    model.C_ChegadaDentroDaJanelaDeTempo = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        V_AT[i]
                                                            <=
                                                        P_FimDoHorizonte * sum(V_Y[i,h] for h in S_Aeronaves)) 
    # 4.16 C_TipoDeAtraso - ok - dc1
    model.C_TipoDeAtraso = pyo.Constraint(S_VoosTabelaComitiva,
                                                        rule = lambda model, i0C: 
                                                        V_BI[i0C] + V_BII[i0C]
                                                        <=
                                                        1)
    # 4.17 C_ValorDoAtraso -ok - dc2
    model.C_ValorDoAtraso = pyo.Constraint(S_VoosTabelaComitiva,
                                                        rule = lambda model, i0C: 
                                                        V_D[i0C]
                                                        <=
                                                        P_AtrasoLimite1 * V_BI[i0C] + P_AtrasoLimite2 * V_BII[i0C]
                                                        )          
    # 4.18 C_AtivacaoDaPrecedencia1 - ok - pc1
    model.C_AtivacaoDaPrecedencia1 = pyo.Constraint(S_VoosReais, S_VoosReais,
                                                        rule = lambda model, i,j: 
                                                        V_DT[j]
                                                        -
                                                        V_DT[i]
                                                        <=
                                                        P_FimDoHorizonte*(V_Z[i,j] + 2 - sum(V_Y[i,h] for h in S_Aeronaves) - sum(V_Y[j,h] for h in S_Aeronaves))
                                                        if i != j else pyo.Constraint.Skip # garante que a restrição só será criada se i != j 
                                                    )

    # 4.19 C_ApenasUmaPrecedencia - ok - pc2
    model.C_ApenasUmaPrecedencia = pyo.Constraint(S_VoosReais, S_VoosReais,
                                                        rule = lambda model, i,j: 
                                                        V_Z[i,j] + V_Z[j,i]
                                                        <=
                                                        1
                                                        if i != j else pyo.Constraint.Skip # garante que a restrição só será criada se i != j 
                                                    )
    # 4.20 C_AtivacaoDaPrecedencia2 - ok - pc3
    model.C_AtivacaoDaPrecedencia2 = pyo.Constraint(S_VoosReais, S_VoosReais,
                                                        rule = lambda model, i,j: 
                                                        V_Z[i,j] + V_Z[j,i]
                                                        >=
                                                        sum(V_Y[i,h] for h in S_Aeronaves) + sum(V_Y[j,h] for h in S_Aeronaves) - 1
                                                        if i != j else pyo.Constraint.Skip # garante que a restrição só será criada se i != j 
                                                    )
    # 4.21 C_DesigualdadeValida - ok - pc4
    model.C_DesigualdadeValida = pyo.Constraint(S_VoosReais,
                                                        rule = lambda model, i: 
                                                        sum(V_Z[i,j] for j in S_VoosReais)
                                                        <=
                                                        len(S_VoosTodos) * sum(V_Y[i,h] for h in S_Aeronaves)
                                                    )               

    # 4.22 C_TempoDeSeguranca - ok - oc1
    model.C_TempoDeSeguranca = pyo.Constraint(S_VoosReais,S_VoosReais,
                                                        rule = lambda model, i,j: 
                                                        V_DT[j] - V_DT[i]
                                                        >=
                                                        P_TempoDeSeguranca * V_Z[i,j] - P_FimDoHorizonte * (1 - V_Z[i,j])
                                                        if i != j else pyo.Constraint.Skip
                                                    )
    # 4.23 C_TempoDePermanencia - ok - oc2
    model.C_TempoDePermanencia = pyo.Constraint(S_VoosReais,S_VoosReais,S_UM,
                                                        rule = lambda model, i,j,um: 
                                                        V_DT[j] - V_DT[i]
                                                        >=
                                                        P_TempoDePermanencia[i] * V_Z[i,j] - P_FimDoHorizonte * (1 - V_Z[i,j])
                                                        if ((i != j) and (P_UMDestino[i] == P_UMDestino[j])) else pyo.Constraint.Skip
                                                    )
    # Voo i incompatível com aeronave h para V_x
    for i in S_VoosReais:
        for h in S_Aeronaves:
            for j in S_VoosReais:
                if P_Compatibilidade[i,h] == 0:
                    V_X[i,j,h].fix(0)

    # Voo j incompatível com aeronave h
    for j in S_VoosReais:
        for h in S_Aeronaves:
            for j in S_VoosReais:
                if P_Compatibilidade[j,h] == 0:
                    V_X[i,j,h].fix(0)

    # Voo i incompatível com aeronave h para V_Y
    for i in S_VoosReais:
        for h in S_Aeronaves:
            if P_Compatibilidade[i,h] == 0:
                V_Y[i,h].fix(0)

    # Respeitar precedência de voos

    for i0 in S_VoosTabela:
        for itransf in S_VoosTransferencia:
    #       for um in S_UM:
                if P_UMDestino[i0] == P_UMDestino[itransf] and i0 != itransf :
                    V_Z[i0,itransf].fix(0)      

    # Informações do Modelo
    # Count all variables in the model
    P_total_variables = len(list(model.component_data_objects(Var)))

    # Count integer variables
    P_integer_variables = len([v for v in model.component_data_objects(Var) if v.domain == Integers])

    # Count binary variables
    P_binary_variables = len([v for v in model.component_data_objects(Var) if v.domain == Binary])

    # Count all constraints in the model
    P_total_constraints = len(list(model.component_data_objects(Constraint)))

    # Print the results
    print(f"Total number of variables: {P_total_variables}")
    print(f"Total number of integer variables: {P_integer_variables}")
    print(f"Total number of binary variables: {P_binary_variables}")
    print(f"Total number of constraints: {P_total_constraints}")

    #opt = SolverFactory(solver_selecionado, executable='C:\\Users\\mateu\\Desktop\\Estudos\\Solvers\\SCIP\\scip-9.2.0\\SCIPOptSuite-9.2.0-win64.exe')
    opt = SolverFactory(solver_selecionado)

    # Verificando se o solver selecionado existe
    #print(opt.available())

    # Nome da rodada
    Ps_NomeDaRodada[instancia] = Ps_NomeDaInstancia + "_" + solver_selecionado + "_" + f"{P_HoraDaRodadaTimeStamp}"

    # Definindo as opções do solver
    # Set solver options for GLPK
    if solver_selecionado == 'glpk':
        opt.options['tmlim'] = 3600  # Set time limit (seconds)
        opt.options['mipgap'] = 0.00  # Set optimality gap (percentage)
        #opt.options['threads'] = 4  # Set the number of threads
        #solver.options['log'] = f"{Ps_NomeDaRodada}.txt"  # Optional: Log file (GLPK doesn't directly support this in Pyomo)

    # CBC
    if solver_selecionado == 'cbc':
        opt.options['seconds'] = 3600  # Set a time limit (optional)
        opt.options['ratio'] = 0.00  # Set optimality gap (optional)
        opt.options['threads'] = 4  # set number of threads (optional)
        #opt.options['logFile'] = f"{Ps_NomeDaRodada}.txt" # escreve o log da rodada

    #Gurobi
    if solver_selecionado == 'gurobi':
        opt.options['TimeLimit'] = 3600  # Set a time limit (optional)
        opt.options['MIPGap'] = 0.00  # Set optimality gap (optional)
        opt.options['Threads'] = 4  # set number of threads (optional)
        #opt.options['LogFile'] = f"{Ps_NomeDaRodada}.txt" # escreve o log da rodada

    # CPLEX
    if solver_selecionado == 'cplex':
        opt.options['TimeLimit'] = 3600  # Set a time limit (optional)
        opt.options['mip_tolerances_mipgap'] = 0.00  # Set optimality gap (optional)
        opt.options['threads'] = 4 # set number of threads (optional)
        #opt.options['LogFile'] = f"{Ps_NomeDaRodada}.txt" # escreve o log da rodada

    # HiGHS
    if (solver_selecionado == 'appsi_highs') or (solver_selecionado == 'highs'):
        opt.options['time_limit'] = 3600  # Set a time limit (optional)
        opt.options['mip_rel_gap'] = 0.00  # Set optimality gap (optional)
        opt.options['threads'] = 4 # set number of threads (optional)
        #opt.options['log_file'] = f"{Ps_NomeDaRodada}.txt" # escreve o log da rodada

    # MOSEK
    if solver_selecionado == 'mosek':
        # Set logging file (this is for MOSEK output)
        #opt.options['logfile'] = f"{Ps_NomeDaRodada}.txt"  # Log file path
        # Set time limit (in seconds)
        opt.options['dparam.optimizer_max_time'] = 3600  # 1 hour limit
        # Set the optimality gap (optional)
        opt.options['dparam.mio_rel_gap_const'] = 0.00001  # 0% gap for exact solution
        # Set number of threads (optional)
        opt.options['iparam.num_threads'] = 4  # Set to 4 threads
    
    # SCIP
    if solver_selecionado == 'scip':       
        # Set SCIP options
        opt.options['limits/time'] = 3600         # Set time limit to 3600 seconds
        opt.options['limits/gap'] = 0.00         # Set optimality gap
        opt.options['parallel/maxnthreads'] = 4  # Use 4 threads

    # Solving the model
    # Relaxação linear
    if relaxacao_linear == 1:
        xfrm = TransformationFactory('core.relax_integer_vars')
        relaxed_model = xfrm.apply_to(model)
    
    start_time[instancia] = datetime.now()
    results = opt.solve(model, tee=True)   
    end_time[instancia] = datetime.now()
    # Resultados
    print("Resultados da rodada: ", Ps_NomeDaRodada, f"Para o solver {solver_selecionado}")
    # Retrieve the objective value of the incumbent solution (best integer solution found)
    P_SOL_incumbent_solution[instancia] = model.obj()

    # Retrieve solver results
    P_SOL_solver_status[instancia] = results.solver.status
    #P_SOL_termination_condition[instancia] = results.solver.termination_condition
    P_SOL_solver_time[instancia] = results.solver.time if hasattr(results.solver, 'time') else None # Time spent solving the model    
    
    # Retrieve the best bound (only for MIP problems)
    #P_SOL_best_bound = results.solver.best_bound if hasattr(results.solver, 'best_bound') else None

    # Retrieve the optimality gap (only for MIP problems)
   # P_SOL_optimality_gap = results.solver.optimality_gap if hasattr(results.solver, 'optimality_gap') else None

   # P_SOL_optimality_gap = results.solver.get('mipgap', None)

    # Output the results
    print(f"Incumbent Solution: {P_SOL_incumbent_solution[instancia]}")
    #print(f"Best Bound: {P_SOL_best_bound}")
    #print(f"Optimality Gap: {P_SOL_optimality_gap}")
    print(f"Solver Status: {P_SOL_solver_status[instancia]}")
    #print(f"Termination Condition: {P_SOL_termination_condition}")
    print(f"Solver Time: {P_SOL_solver_time[instancia]} seconds")     

    elapsed_time[instancia] = end_time[instancia] - start_time[instancia]                                      

<string>:14: SyntaxWarning: invalid escape sequence '\{'
<>:14: SyntaxWarning: invalid escape sequence '\{'
<>:14: SyntaxWarning: invalid escape sequence '\M'
<string>:14: SyntaxWarning: invalid escape sequence '\{'
<>:14: SyntaxWarning: invalid escape sequence '\{'
<>:14: SyntaxWarning: invalid escape sequence '\M'
C:\Users\mateu\AppData\Local\Temp\ipykernel_37608\2872397629.py:14: SyntaxWarning: invalid escape sequence '\{'
  Ps_CaminhoDaPlanilha = f"G:\Meu Drive\Planilhas para Estudos\{instancia}.xlsx"
C:\Users\mateu\AppData\Local\Temp\ipykernel_37608\2872397629.py:14: SyntaxWarning: invalid escape sequence '\M'
  Ps_CaminhoDaPlanilha = f"G:\Meu Drive\Planilhas para Estudos\{instancia}.xlsx"


[5, 6, 9, 12, 13, 14, 19, 21, 26, 28, 29, 33, 34, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 50]
Ps_NomeDaInstancia =  i50_Simulado
'Any'. The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend the
domain of this Paramto be 'Any', you can suppress this warning by explicitly
specifying 'within=Any' to the Param constructor.  (deprecated in 5.6.9, will
be removed in (or after) 6.0) (called from c:\Users\mateu\anaconda3\Lib\site-
packages\pyomo\core\base\indexed_component.py:714)
P_DuracaoDoVoo : Size=50, Index=S_VoosReais, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :    96
      2 :   117
      3 :    91
      4 :   135
      5 :   107
      6 :    41
      7 :   118
      8 :   113
      9 :    84
     10 :    92
     11 :   103
     12 :    89
     13 :    85
     14 :    92
     15 :   106
     16 :   108
     17 :    78
     18 :    81
     19 :   112
     20 :   103
     21 :    

C:\Users\mateu\AppData\Local\Temp\ipykernel_37608\2872397629.py:14: SyntaxWarning: invalid escape sequence '\{'
  Ps_CaminhoDaPlanilha = f"G:\Meu Drive\Planilhas para Estudos\{instancia}.xlsx"
C:\Users\mateu\AppData\Local\Temp\ipykernel_37608\2872397629.py:14: SyntaxWarning: invalid escape sequence '\M'
  Ps_CaminhoDaPlanilha = f"G:\Meu Drive\Planilhas para Estudos\{instancia}.xlsx"


RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "highs_appsi"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: highs_appsi
	type: highs_appsi
	_args: ()
	options: {}

# Exportação

In [ ]:
# Exportação
df_export = pd.DataFrame({"P_SOL_NomeDaRodada": Ps_NomeDaRodada,
                            "P_SOL_incumbent_solution": P_SOL_incumbent_solution, 
                          "P_SOL_solver_status": P_SOL_solver_status, 
                          "P_SOL_solver_time": P_SOL_solver_time,
                          "elapsed_time" : elapsed_time
                          #"P_SOL_best_bound": P_SOL_best_bound,
                          #"P_SOL_optimality_gap": P_SOL_optimality_gap
                          })
print(df_export)

for i in S_VoosReais:
    for h in S_Aeronaves:
        if V_Y[i,h].value > 0:
            print(f"V_Y({i},{h}) = ", V_Y[i,h].value)

#df_export.to_excel(f"DataFram_{Ps_NomeDaRodada}.xlsx", index = True)
#df_export.to_excel(f"G:\Meu Drive\Planilhas para Estudos\ResultadosPyomoRL\RelaxacaoLinear_{solver_selecionado}_{P_HoraDaRodadaTimeStamp}.xlsx", index = True)